In [1]:
import pandas as pd

### 2019 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2019_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2018
start_year = 2019
target_year = 2020

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [3]:
# Initialize an empty DataFrame for the master data
master_2019 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.startswith("HRM2_KT_"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=3)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'HRM 2': 'HRM2-ID',
            'Unnamed: 2': 'HRM2-Bezeichnung',
            'Rechnung': f'Rechnung {previous_year}',
            'Budget': f'Budget {start_year}',
            'Anpassung Budget**': f'Anpassung Budget {start_year}',
            'Budgetplus***': f'Budgetplus {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget.1': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Filter the DataFrame to keep only rows where 'Referenz-ID' contains 'HRM2'
        df = df[df['Referenz-ID'].str.contains('HRM2', na=False)]

        # Add a new column with the DataFrame name (sheet name)
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split('HRM2_KT_').str[1]

        df['Year'] = 2019

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2019 = pd.concat([master_2019, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2019.head()

,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2018,Budget 2019,Rechnung 2019,Budget 2020,Source,Kanton,Year,Compte,Compte.1
0,HRM2_ER0030,30,Personalaufwand,24420046.28348,24930238.94484,25078006.42584,25365050.64956,HRM2_KT_alle,alle,2019,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,9405757.49866,9515307.06695,9695994.15067,10070494.85185,HRM2_KT_alle,alle,2019,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1096543.31872,1072231.29400,1093416.79827,1130573.04500,HRM2_KT_alle,alle,2019,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,113993.91864,47188.05900,124425.54987,48634.50300,HRM2_KT_alle,alle,2019,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2928793.02640,2855453.26284,2834876.47063,2894989.36082,HRM2_KT_alle,alle,2019,NaN,NaN


Translating FR columns to DE

In [4]:
# Display rows where the column 'Kanton' is 'HRM2_KT_VD'
filtered_rows = master_2019[master_2019['Kanton'] == 'HRM2_KT_VD']

# Display the filtered rows
filtered_rows.head()

,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2018,Budget 2019,Rechnung 2019,Budget 2020,Source,Kanton,Year,Compte,Compte.1


In [5]:
# 1. Update 'Rechnung 2018' based on 'Compte'
master_2019['Rechnung 2018'] = master_2019.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2018']) and not pd.isna(row['Compte']) else row['Rechnung 2018'],
    axis=1
)

# 2. Update 'Rechnung 2019' based on 'Compte.1'
master_2019['Rechnung 2019'] = master_2019.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2019']) and not pd.isna(row['Compte.1']) else row['Rechnung 2019'],
    axis=1
)

# Display the first few rows of the updated DataFrame
master_2019.head()


,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2018,Budget 2019,Rechnung 2019,Budget 2020,Source,Kanton,Year,Compte,Compte.1
0,HRM2_ER0030,30,Personalaufwand,24420046.28348,24930238.94484,25078006.42584,25365050.64956,HRM2_KT_alle,alle,2019,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,9405757.49866,9515307.06695,9695994.15067,10070494.85185,HRM2_KT_alle,alle,2019,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1096543.31872,1072231.29400,1093416.79827,1130573.04500,HRM2_KT_alle,alle,2019,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,113993.91864,47188.05900,124425.54987,48634.50300,HRM2_KT_alle,alle,2019,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2928793.02640,2855453.26284,2834876.47063,2894989.36082,HRM2_KT_alle,alle,2019,NaN,NaN


In [6]:
# Display rows where the column 'Kanton' is 'HRM2_KT_VD'
filtered_rows = master_2019[master_2019['Kanton'] == 'HRM2_KT_VD']

# Display the filtered rows
filtered_rows.head()

,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2018,Budget 2019,Rechnung 2019,Budget 2020,Source,Kanton,Year,Compte,Compte.1


In [7]:
len(master_2019)

5265

In [8]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2019_data.csv'

# Save the DataFrame to a CSV file
master_2019.to_csv(output_path, index=False)
